In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding, LSTM

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

# encoding은 보통 utf-8, cp949 로 하면되지만 이번 파일은 latin1
spam_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ai_school/data/spam.csv', encoding='latin1')

# 3, 4, 5열 삭제 후 컬럼명 변경
spam_data = spam_data.dropna(axis=1)
spam_data.columns = ["label", "mail"]

# ham, spam 숫자로 변경
spam_data['label'] = spam_data['label'].replace('spam', 1)
spam_data['label'] = spam_data['label'].replace('ham', 0)

# 단어 아니면 삭제
spam_data['mail'] = spam_data['mail'].str.replace("[^\w]", " ") # str.replace를 해야 정규표현식 사용 가능

# 혹시나 공백이 있으면
spam_data['mail'] = spam_data['mail'].replace('', np.nan)
spam_data['label'] = spam_data['label'].replace('', np.nan)

# 결측치 있으면 모두 제거
spam_data = spam_data.dropna(how='any')

print("# preprocessing done")


# test/train 스플릿하고
mail_train, mail_test, y_train, y_test = train_test_split(spam_data['mail'], spam_data['label'], test_size=0.2, shuffle=False)

print('# split done')


stopwords = ['a', 'an']

# 토큰화 진행
X_train = []
for stc in mail_train:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in mail_test:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)

print('# tokenization done')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# X_train 단어들을 토대로 정수 인덱스 설정
# 빈도수가 높은 것부터 4000개만 정수 인덱스로 변환하겠다!
tokenizer = Tokenizer(4000)
tokenizer.fit_on_texts(X_train)

# 위에서 설정된 정수 인덱스를 토대로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print('# int_encoding done')

In [ ]:
print(len(tokenizer.word_index)) # 전체 단어 갯수

low_count = 0
for word, word_count in tokenizer.word_counts.items():
    if word_count == 1:
        low_count += 1
print(low_count)

In [ ]:
max_length = 0
for data in X_train:
    if max_length < len(data):
        max_length = len(data)
print(max_length)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 패딩 작업 진행
# 대부분 40~50의 길이를 갖기 때문에, 원래의 max_length가 아닌 50으로 지정하는 것이 효율적.
max_len = 50
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(4000, 32)) # 최대 128이지만, 데이터셋이 단순한 편이라 32로 지정
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

In [ ]:
sentence = input()
# 토큰화
token_stc = sentence.split()
# 정수 인코딩
encode_stc = tokenizer.texts_to_sequences([token_stc])
# 패딩
pad_stc = pad_sequences(encode_stc, maxlen = 50)

score = model.predict(pad_stc)
print(score)